# Create and train model

In [ ]:
!pip install datasets

In [1]:
import joblib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report
from datasets import load_dataset
dataset = load_dataset('wili_2018')

Reusing dataset wili_2018 (/root/.cache/huggingface/datasets/wili_2018/WiLI-2018 dataset/1.1.0/78d7fe4a9d0a01168e45657f302c776ee0afc0978d44e2c3759f4c4975b845f5)


  0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
_CLASSES=["cdo","glk","jam","lug","san","rue","wol","new","mwl","bre","ara","hye","xmf","ext","cor","yor","div","asm","lat","cym","hif","ace","kbd","tgk","rus","nso","mya","msa","ava","cbk","urd","deu","swa","pus","bxr","udm","csb","yid","vro","por","pdc","eng","tha","hat","lmo","pag","jav","chv","nan","sco","kat","bho","bos","kok","oss","mri","fry","cat","azb","kin","hin","sna","dan","egl","mkd","ron","bul","hrv","som","pam","nav","ksh","nci","khm","sgs","srn","bar","cos","ckb","pfl","arz","roa-tara","fra","mai","zh-yue","guj","fin","kir","vol","hau","afr","uig","lao","swe","slv","kor","szl","srp","dty","nrm","dsb","ind","wln","pnb","ukr","bpy","vie","tur","aym","lit","zea","pol","est","scn","vls","stq","gag","grn","kaz","ben","pcd","bjn","krc","amh","diq","ltz","ita","kab","bel","ang","mhr","che","koi","glv","ido","fao","bak","isl","bcl","tet","jpn","kur","map-bms","tyv","olo","arg","ori","lim","tel","lin","roh","sqi","xho","mlg","fas","hbs","tam","aze","lad","nob","sin","gla","nap","snd","ast","mal","mdf","tsn","nds","tgl","nno","sun","lzh","jbo","crh","pap","oci","hak","uzb","zho","hsb","sme","mlt","vep","lez","nld","nds-nl","mrj","spa","ceb","ina","heb","hun","que","kaa","mar","vec","frp","ell","sah","eus","ces","slk","chr","lij","nep","srd","ilo","be-tarask","bod","orm","war","glg","mon","gle","min","ibo","ile","epo","lav","lrc","als","mzn","rup","fur","tat","myv","pan","ton","kom","wuu","tcy","tuk","kan","ltg"]
LABELS_STOI = {k: v for k, v in enumerate(_CLASSES)}
LABELS_IOST = {v: k for k, v in enumerate(_CLASSES)}

In [103]:
print(LABELS_IOST["deu"])
print(LABELS_STOI[24])

31
rus


In [108]:
supported_lang = ["bos", "bul", "ces", "dan", "deu", "ell", "eng", "fra", "hrv", "ita", "lav", "nld", "pol", "por", "ron", "rus", "slk", "slv", "spa", "srp", "swe", "tur"]
lang_label = [LABELS_IOST[lang] for lang in supported_lang]
lang_label.sort()

In [127]:
df_train = pd.DataFrame(dataset['train'])
df_test = pd.DataFrame(dataset['test'])
df_res = df_train.append(df_test)
df_train, df_test = train_test_split(df_res, test_size=0.2)

In [128]:
df_train = df_train[df_train['label'].isin(lang_label)]
df_test = df_test[df_test['label'].isin(lang_label)]

In [129]:
def preprocess(txt):
  txt = str(txt)
  return ''.join(e for e in txt if e.isalpha() or e == " ")

In [130]:
df_train['sentence'] = df_train['sentence'].apply(preprocess)
df_test['sentence'] = df_test['sentence'].apply(preprocess)

In [151]:
X_train, X_test, y_train, y_test = df_train['sentence'], df_test['sentence'], df_train['label'], df_test['label']
pipeline = Pipeline([
   ('vectorizer', CountVectorizer(analyzer = 'char',ngram_range=(3,3))),  
   ('model', MultinomialNB())
])
pipeline.fit(X_train,y_train)
y_pred = pipeline.predict(X_test)
accuracy_score(y_test, y_pred)

0.9661209640745794

In [163]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

          24       1.00      1.00      1.00       208
          31       0.96      0.97      0.97       213
          39       0.98      0.97      0.98       195
          41       0.80      0.99      0.89       190
          52       0.85      0.81      0.83       194
          62       0.99      0.98      0.99       188
          65       1.00      0.99      0.99       188
          66       1.00      0.97      0.99       188
          67       0.82      0.87      0.85       201
          82       0.96      0.98      0.97       194
          93       1.00      1.00      1.00       213
          94       1.00      0.96      0.98       213
          97       1.00      0.91      0.95       190
         107       0.96      0.98      0.97       182
         111       1.00      1.00      1.00       218
         126       0.98      0.95      0.97       212
         185       1.00      0.97      0.98       195
         188       0.99    

In [171]:
joblib.dump(pipeline, 'langdetect.pkl')
!cp langdetect.pkl '/content/drive/MyDrive/Colab Notebooks/rasa/models/'

# Run model

In [2]:
_CLASSES=["cdo","glk","jam","lug","san","rue","wol","new","mwl","bre","ara","hye","xmf","ext","cor","yor","div","asm","lat","cym","hif","ace","kbd","tgk","rus","nso","mya","msa","ava","cbk","urd","deu","swa","pus","bxr","udm","csb","yid","vro","por","pdc","eng","tha","hat","lmo","pag","jav","chv","nan","sco","kat","bho","bos","kok","oss","mri","fry","cat","azb","kin","hin","sna","dan","egl","mkd","ron","bul","hrv","som","pam","nav","ksh","nci","khm","sgs","srn","bar","cos","ckb","pfl","arz","roa-tara","fra","mai","zh-yue","guj","fin","kir","vol","hau","afr","uig","lao","swe","slv","kor","szl","srp","dty","nrm","dsb","ind","wln","pnb","ukr","bpy","vie","tur","aym","lit","zea","pol","est","scn","vls","stq","gag","grn","kaz","ben","pcd","bjn","krc","amh","diq","ltz","ita","kab","bel","ang","mhr","che","koi","glv","ido","fao","bak","isl","bcl","tet","jpn","kur","map-bms","tyv","olo","arg","ori","lim","tel","lin","roh","sqi","xho","mlg","fas","hbs","tam","aze","lad","nob","sin","gla","nap","snd","ast","mal","mdf","tsn","nds","tgl","nno","sun","lzh","jbo","crh","pap","oci","hak","uzb","zho","hsb","sme","mlt","vep","lez","nld","nds-nl","mrj","spa","ceb","ina","heb","hun","que","kaa","mar","vec","frp","ell","sah","eus","ces","slk","chr","lij","nep","srd","ilo","be-tarask","bod","orm","war","glg","mon","gle","min","ibo","ile","epo","lav","lrc","als","mzn","rup","fur","tat","myv","pan","ton","kom","wuu","tcy","tuk","kan","ltg"]
LABELS_STOI = {k: v for k, v in enumerate(_CLASSES)}
LABELS_IOST = {v: k for k, v in enumerate(_CLASSES)}

In [8]:
import joblib
drive = '/content/drive/MyDrive/Colab Notebooks/rasa/models/'
model = joblib.load(drive + 'langdetect.pkl')
x = "habe eine Frage"
y = LABELS_STOI[model.predict([x])[0]]
prob = model.predict_proba([x]).flatten()[1]
print("P('deu'|x) = " + str(prob))
print("most likly class: " + y)

P('deu'|x) = 0.9999397191843299
most likly class: deu
